# Analyse de la Volatilité de l'EUR/USD par plages horaires
Ce notebook analyse la volatilité absolue de la paire EUR/USD à partir de données OHLC (Open, High, Low, Close, Volume).

- Source : données minute de Dukascopy en format GMT
- La conversion vers le fuseau Europe/Paris est intégrée (UTC+1 ou UTC+2 avec heure d'été)
- La colonne `Close` est utilisée comme référence de prix
- La volatilité est mesurée sur trois plages horaires : 8h-11h, 11h-13h (lunch), et 13h-17h


In [2]:
# Imports
import pandas as pd
import numpy as np


In [12]:
# Chargement des données CSV OHLC téléchargées depuis Dukascopy
df = pd.read_csv("eurusd.csv")

#Nettoyage de la chaîne de dates
df["Gmt time"]= df["Gmt time"].str.replace(" ", " ") #double espace-> simple
df["Gmt time"]= df["Gmt time"]. str.strip()          #trim

#Conversion de la colonne "Gmt time" en datetime
df["Gmt time"]=pd.to_datetime(df["Gmt time"], format="%d.%m.%Y %H:%M:%S.%f")

#Renommage et indexation
df = df.rename(columns={"Gmt time": "datetime"})
df = df.set_index("datetime")

# Convertir le fuseau horaire de GMT vers Europe/Paris (UTC+1 ou UTC+2)
df.index=df.index.tz_localize("Gmt").tz_convert("Europe/Paris")

#Création de la colonne de prix (basée sur la clôture).                                                 
df["price"] = df["Close"]

df.head()

,Open,High,Low,Close,Volume,price
datetime,,,,,,
2024-01-01 01:00:00+01:00,1.10374,1.10374,1.10374,1.10374,0.0,1.10374
2024-01-01 01:01:00+01:00,1.10374,1.10374,1.10374,1.10374,0.0,1.10374
2024-01-01 01:02:00+01:00,1.10374,1.10374,1.10374,1.10374,0.0,1.10374
2024-01-01 01:03:00+01:00,1.10374,1.10374,1.10374,1.10374,0.0,1.10374
2024-01-01 01:04:00+01:00,1.10374,1.10374,1.10374,1.10374,0.0,1.10374


In [17]:
# Fonction pour calculer la volatilité absolue sur une plage horaire spécifique
def compute_intraday_volatility(df, start_hour, end_hour):
    return df.between_time(f"{start_hour}:00", f"{end_hour}:00")["price"].resample("1D").agg(lambda x: x.max() - x.min())

# Calcul de la volatilité pour chaque plage horaire
vol_8_11 = compute_intraday_volatility(df, 8, 11)
vol_11_13 = compute_intraday_volatility(df, 11, 13)
vol_13_17 = compute_intraday_volatility(df, 13, 17)

# Fusion des résultats dans un tableau unique
vol_df = pd.concat([vol_8_11, vol_11_13, vol_13_17], axis=1)
vol_df.columns = ["Volatilité 8-11h", "Volatilité 11-13h", "Volatilité 13-17h"]
vol_df.dropna(inplace=True)
vol_df.head()

,Volatilité 8-11h,Volatilité 11-13h,Volatilité 13-17h
datetime,,,
2024-01-01 00:00:00+01:00,0.00000,0.00000,0.00000
2024-01-02 00:00:00+01:00,0.00243,0.00486,0.00284
2024-01-03 00:00:00+01:00,0.00290,0.00212,0.00379
2024-01-04 00:00:00+01:00,0.00387,0.00233,0.00319
2024-01-05 00:00:00+01:00,0.00288,0.00132,0.01175


In [26]:
vol_df.to_csv("volatilite_eurusd.csv")